In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d phylake1337/fire-dataset

!unzip -q fire-dataset.zip -d fire-dataset

Saving kaggle.json to kaggle.json
Dataset URL: https://www.kaggle.com/datasets/phylake1337/fire-dataset
License(s): CC0-1.0
100% 387M/387M [00:03<00:00, 144MB/s]
100% 387M/387M [00:03<00:00, 108MB/s]


In [3]:
import os
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

class1_dir = '/content/fire-dataset/fire_dataset/fire_images'
class2_dir = '/content/fire-dataset/fire_dataset/non_fire_images'
output_dir = '/content/drive/MyDrive/Fire_Aug'

os.makedirs(os.path.join(output_dir, 'fire'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'nonfire'), exist_ok=True)

In [4]:
def load_and_resize_images(input_dir, size=(128, 128)):
    images = []
    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        img = Image.open(img_path).convert('RGB')
        img = img.resize(size)
        images.append(np.array(img))
    return np.array(images)

In [5]:
class1_images = load_and_resize_images(class1_dir)

In [6]:
coun = 0
for i, img in enumerate(class1_images):
  coun += 1
  img = Image.fromarray(img)
  img.save(os.path.join(output_dir, 'fire', f'resized_{i}.png'))
print(coun)

755


In [7]:
class2_images = load_and_resize_images(class2_dir)

In [11]:
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
augmented_images = []

for img in class2_images:
  img = img.reshape((1,)+img.shape)
  cou = 0
  for bth in datagen.flow(img,batch_size=1):
    augmented_images.append(bth[0].astype('uint8'))
    cou += 1
    if cou >= 4: break

for i, img in enumerate(augmented_images):
  img = Image.fromarray(img)
  img.save(os.path.join(output_dir, 'nonfire', f'augmented_{i}.png'))
print(len(augmented_images))
print("Data augmentation and resizing completed!")

976
Data augmentation and resizing completed!


In [10]:
os.makedirs(os.path.join(output_dir, 'nonfire'), exist_ok=True)